In [1]:
from src.dataloader import *

from torch.utils.data import DataLoader
import matplotlib.pyplot as plt
from torchvision import transforms, datasets
import torch.nn as nn


import tensorflow as tf
import keras
import keras.layers as layers

In [2]:
tf.random.set_seed(42)

# Import des données

## Terminologie: Définition des hyperparamètres du modèle

* Hyperparamètre = Paramètre qui contrôle le processus d'apprentissage de notre réseau de neurones.

* Batch Size = Hyperparamètre qui définit le nombres d'images traitées par notre algorithme avant de mettre à jour les paramètres du modèle (ses poids et ses biais). Plus le batck size est élevé, plus il faut de mémoire. En général, on utilise des batchs de taille 64, 128, 256 ou 512.

* Epoch = Hyperparamètre qui définit le nombre d'itérations effectuées sur toutes les données d'apprentissage pour entraîner notre modèle.

* Learning rate = Hyperparamètre qui contrôle la taille du pas effectué quand on se déplace vers le minimum de la fonction de perte. Si sa valeur est trop importante on risque d'osciller autour du minimum sans converger vers sa valeur et si le learning rate est trop faible la convergence .

Exemple: si j'ai 1000 individus dans l'échantillon d'apprentissage, que batch_size = 500 et epoch = 2.
* Première étape : On tire 500 individus dans l'échantillon d'apprentissage et on met à jour les paramètres de notre modèle.
* Deuxième étape : On tire les 500 derniers individus de l'échantillon d'apprentissage et on met à jour les paramètres de notre modèle. La première itération du paramètre epoch est achevée.
* Troisième étape : On tire à nouveau 500 individus dans l'échantillon d'apprentissage et on met à jour les paramètres de notre modèle (ces individus avaient déjà été tirés tirés dans l'étape un ou deux).
* Quatrième étape : On tire les 500 individus de l'échantillon d'apprentissage qui n'ont pas été tirés dans l'étape 3 et on met à jour les paramètres de notre modèle. La seconde itération de l'hyperparamètre epoch est achevée.

Ainsi, dans notre exemple, il faut deux itérations (deux batchs) pour réaliser un epoch.

In [12]:
label_attribution=LabelAttribution(path_image_google="C:/Users/yanis/OneDrive/Documents/Projet Stat/Données/bdappv/google/img/", 
                                   path_mask_google='C:/Users/yanis/OneDrive/Documents/Projet Stat/Données/bdappv/google/mask/',
                                   path_metadata='C:/Users/yanis/OneDrive/Documents/Projet Stat/Données/bdappv/metadata.csv',
                                   colonne_identifiant='identifiant',
                                   path_export_train_test="C:/Users/yanis/OneDrive/Documents/Projet Stat/statapps",
                                   path_image_ign='C:/Users/yanis/OneDrive/Documents/Projet Stat/Données/bdappv/ign/img',
                                   path_mask_ign='C:/Users/yanis/OneDrive/Documents/Projet Stat/Données/bdappv/ign/mask/',
                                   use_img_google=True,
                                   use_img_ign=False
                                    )

In [13]:
label_attribution.run()

Le LeNET5 a été conçu pour prendre en entrée des images de dimension 28*28. On passe donc les images 400 x 400 en 28 x 28. On convertit ensuite ces dernières en tenseurs. 

In [14]:
path_train="C:/Users/yanis/OneDrive/Documents/Projet Stat/statapps/train_data.csv"
path_test="C:/Users/yanis/OneDrive/Documents/Projet Stat/statapps/test_data.csv"

transformed_train_dataset  = CustomImageDataset(path_train,"C:/Users/yanis/OneDrive/Documents/Projet Stat/Données/bdappv/google/img/", transform=transforms.Compose([
                                               transforms.Resize(28),
                                               transforms.ToTensor(),
                                           ]))
transformed_test_dataset = CustomImageDataset(path_test,"C:/Users/yanis/OneDrive/Documents/Projet Stat/Données/bdappv/google/img/",
                                            transform=transforms.Compose([
                                               transforms.Resize(28),
                                               transforms.ToTensor(),
                                           ]))

In [15]:
print("Nombre d'images dans le train: {}".format(transformed_train_dataset.__len__()))
print("Nombre d'images dans le test: {}".format(transformed_test_dataset.__len__()))

Nombre d'images dans le train: 23045
Nombre d'images dans le test: 5762


In [16]:
train_dataloader = DataLoader(transformed_train_dataset, batch_size=23045, shuffle=True)
test_dataloader = DataLoader(transformed_test_dataset, batch_size=5762, shuffle=True)

In [20]:
# Display image and label.

# train_features, train_labels = next(iter(train_dataloader))

# print(f"Feature batch shape: {train_features.size()}")
# #print(f"Labels batch shape: {train_labels.size()}")
# img = train_features[2].squeeze()
# label = train_labels[2]
# plt.imshow(img.T)
# plt.show()
# print(f"Label: {label}")

On normalise aussi les images: pour chaque chaque channel d'un tenseur, on met la moyenne à 0 et la variance à 1.


output[channel] = (input[channel] - mean[channel]) / std[channel]

In [21]:
mean_train, std_train = mean_std(train_dataloader)
print("Moyenne par channel: {}".format(mean_train.tolist()))
print("Ecart-type par channel: {}".format(std_train.tolist()))

Moyenne par channel: [0.3492567241191864, 0.35663655400276184, 0.307168185710907]
Ecart-type par channel: [0.16620372235774994, 0.14752432703971863, 0.14564631879329681]


In [22]:
mean_test, std_test = mean_std(test_dataloader)
print("Moyenne par channel: {}".format(mean_test.tolist()))
print("Ecart-type par channel: {}".format(std_test.tolist()))

Moyenne par channel: [0.3499918580055237, 0.35766586661338806, 0.30810636281967163]
Ecart-type par channel: [0.16761955618858337, 0.14885342121124268, 0.1473139077425003]


Si on avait utilisé des batchs, on aurait utilisé le code suivant:

In [23]:
# mean_train, std_train = batch_mean_and_sd(train_dataloader)
# print("Moyenne par channel: {}".format(mean_train.tolist()))
# print("Ecart-type par channel: {}".format(std_train.tolist()))


# mean_test, std_test = batch_mean_and_sd(test_dataloader)
# print("Moyenne par channel: {}".format(mean_test.tolist()))
# print("Ecart-type par channel: {}".format(std_test.tolist()))

On normalise les données. On ajoute aussi des rotations horizontales aléatoires. Avec probabilité 0.5, l'image est retournée.

In [24]:
normalized_train_dataset  = CustomImageDataset(path_train,"C:/Users/yanis/OneDrive/Documents/Projet Stat/Données/bdappv/google/img/", transform=transforms.Compose([
                                               transforms.Resize(28),
                                               transforms.ToTensor(),
                                               transforms.Normalize(mean = mean_train.tolist(),
                                                                    std= std_train.tolist()),
                                               transforms.RandomHorizontalFlip(),
                                           ]))

normalized_test_dataset  = CustomImageDataset(path_test,"C:/Users/yanis/OneDrive/Documents/Projet Stat/Données/bdappv/google/img/", transform=transforms.Compose([
                                               transforms.Resize(28),
                                               transforms.ToTensor(),
                                               transforms.Normalize(mean = mean_test.tolist(),
                                                                    std= std_test.tolist()),
                                               transforms.RandomHorizontalFlip(),
                                           ]))

train_dataloader = DataLoader(normalized_train_dataset, batch_size=23045, shuffle=False)
test_dataloader = DataLoader(normalized_test_dataset, batch_size=5762, shuffle=False)

# LeNet5: implémentation et apprentissage

In [31]:
from keras import backend as K

def recall_m(y_true, y_pred):
    true_positives = K.sum(K.round(K.clip(y_true * y_pred, 0, 1)))
    possible_positives = K.sum(K.round(K.clip(y_true, 0, 1)))
    recall = true_positives / (possible_positives + K.epsilon())
    return recall

def precision_m(y_true, y_pred):
    true_positives = K.sum(K.round(K.clip(y_true * y_pred, 0, 1)))
    predicted_positives = K.sum(K.round(K.clip(y_pred, 0, 1)))
    precision = true_positives / (predicted_positives + K.epsilon())
    return precision

def f1_m(y_true, y_pred):
    precision = precision_m(y_true, y_pred)
    recall = recall_m(y_true, y_pred)
    return 2*((precision*recall)/(precision+recall+K.epsilon()))



In [50]:
from keras.losses import BinaryCrossentropy
import tensorflow_addons as tfa

loss_function_used = BinaryCrossentropy()

def buildModel(learnRate=0.01, dropout=0.02):
    model = keras.Sequential()
    # Layer 1 Conv2D
    model.add(layers.Conv2D(filters=6, kernel_size=(5, 5), strides=(1, 1), activation='tanh', input_shape=(28,28,3), padding="same"))
    # Layer 2 Pooling Layer
    model.add(layers.AveragePooling2D(pool_size=(2, 2), strides=(2, 2), padding='valid'))
    # Layer 3 Conv2D
    model.add(layers.Conv2D(filters=16, kernel_size=(5, 5), strides=(1, 1), activation='tanh', padding='valid'))
    # Layer 4 Pooling Layer
    model.add(layers.AveragePooling2D(pool_size=(2, 2), strides=(2, 2), padding='valid'))
    model.add(layers.Flatten())
    model.add(layers.Dense(units=120, activation='tanh'))
    model.add(layers.Dense(units=84, activation='tanh'))
    model.add(layers.Dense(units=2, activation=None))
    model.compile(optimizer='sgd',loss=tf.keras.losses.binary_crossentropy,metrics=["Accuracy", recall_m, precision_m, f1_m])


    #model.compile(loss=loss_function_used, optimizer=tf.keras.optimizers.SGD(learning_rate=0.01), metrics=[keras.metrics.Accuracy(name="Accuracy"),f1_m ,keras.metrics.Precision(name="Precision"), keras.metrics.Recall(name="Recall")])

    return model


In [28]:
X_train, Y_train=next(iter(train_dataloader))
X_test, Y_test=next(iter(test_dataloader))

In [29]:
X_train_permute=X_train.permute(0, 2, 3,1)
X_test_permute=X_test.permute(0, 2, 3,1)

In [57]:
model_init=buildModel()

model_init.fit(x=X_train_permute.numpy(), y=Y_train.numpy(),
	validation_data=(X_test_permute.numpy(), Y_test.numpy()),
	batch_size=8,
	epochs=5)

Epoch 1/5
2881/2881 [==============================] - 25s 8ms/step - loss: 6.6898 - Accuracy: 0.4809 - recall_m: 1.2121 - precision_m: 0.5751 - f1_m: 0.7445 - val_loss: 7.4857 - val_Accuracy: 0.4488 - val_recall_m: 1.0259 - val_precision_m: 0.5459 - val_f1_m: 0.6955
Epoch 2/5
2881/2881 [==============================] - 23s 8ms/step - loss: 6.4928 - Accuracy: 0.4552 - recall_m: 1.2703 - precision_m: 0.5786 - f1_m: 0.7690 - val_loss: 6.5827 - val_Accuracy: 0.4969 - val_recall_m: 1.8110 - val_precision_m: 0.5633 - val_f1_m: 0.8413
Epoch 3/5
2881/2881 [==============================] - 21s 7ms/step - loss: 6.5657 - Accuracy: 0.4594 - recall_m: 1.3071 - precision_m: 0.5838 - f1_m: 0.7731 - val_loss: 4.9193 - val_Accuracy: 0.4295 - val_recall_m: 1.6472 - val_precision_m: 0.6561 - val_f1_m: 0.9161
Epoch 4/5
2881/2881 [==============================] - 20s 7ms/step - loss: 6.3231 - Accuracy: 0.4701 - recall_m: 1.3639 - precision_m: 0.5914 - f1_m: 0.7972 - val_loss: 7.3659 - val_Accuracy: 0.5

In [64]:
loss, accuracy, recall, precision, f1score = model_init.evaluate(X_test_permute.numpy(), Y_test.numpy(), verbose=0)
print("Loss sur l'échantillon test: {}".format(round(loss,3)))
print("Accuracy sur l'échantillon test: {}".format(round(accuracy,3)))
print("Recall sur l'échantillon test: {}".format(round(recall,3)))
print("Precision sur l'échantillon test: {}".format(round(precision,3)))
print("F1-score sur l'échantillon test: {}".format(round(f1score,3)))

Loss sur l'échantillon test: 5.703
Accuracy sur l'échantillon test: 0.48
Recall sur l'échantillon test: 1.592
Precision sur l'échantillon test: 0.618
F1-score sur l'échantillon test: 0.885


# Hyperparamétrisation du modèle: random search

https://pyimagesearch.com/2021/05/31/hyperparameter-tuning-for-deep-learning-with-scikit-learn-keras-and-tensorflow/

In [71]:
learnRate = [1e-2, 1e-3, 1e-4]
dropout = [0.2,0.3, 0.4, 0.5,0.6,0.7,0.8]
batchSize = [4, 8, 16, 32, 64, 128, 256]
epochs = [10, 20, 30, 40,50]

grid = dict(
	learnRate=learnRate,
	dropout=dropout,
	batch_size=batchSize,
	epochs=epochs
)

In [72]:
from keras.wrappers.scikit_learn import KerasClassifier
from sklearn.model_selection import RandomizedSearchCV

model = KerasClassifier(build_fn=buildModel, verbose=1)
#grid = GridSearchCV(estimator=model, param_grid=hyperMatrix)

#n_iter: nombre de modèles entrainés
searcher = RandomizedSearchCV(estimator=model, cv=5, n_iter= 10, param_distributions=grid, scoring='recall')
searchResults = searcher.fit(X_train_permute.numpy(), Y_train.numpy())

Epoch 1/40
1153/1153 [==============================] - 13s 10ms/step - loss: 7.1620 - Accuracy: 0.5112 - recall_m: 1.0278 - precision_m: 0.5409 - f1_m: 0.6973
Epoch 2/40
1153/1153 [==============================] - 10s 9ms/step - loss: 7.2790 - Accuracy: 0.5040 - recall_m: 1.0837 - precision_m: 0.5499 - f1_m: 0.7180
Epoch 3/40
1153/1153 [==============================] - 10s 9ms/step - loss: 6.8185 - Accuracy: 0.5063 - recall_m: 1.0735 - precision_m: 0.5873 - f1_m: 0.7431
Epoch 4/40
1153/1153 [==============================] - 11s 10ms/step - loss: 7.7260 - Accuracy: 0.5037 - recall_m: 0.8522 - precision_m: 0.5306 - f1_m: 0.6309
Epoch 5/40
1153/1153 [==============================] - 10s 9ms/step - loss: 7.7027 - Accuracy: 0.5025 - recall_m: 0.8571 - precision_m: 0.5336 - f1_m: 0.6357
Epoch 6/40
1153/1153 [==============================] - 10s 8ms/step - loss: 7.6924 - Accuracy: 0.5012 - recall_m: 0.8667 - precision_m: 0.5329 - f1_m: 0.6365
Epoch 7/40
1153/1153 [======================

In [73]:
# summarize grid search information
bestScore = searchResults.best_score_
bestParams = searchResults.best_params_
print("[INFO] best score is {:.2f} using {}".format(bestScore,
	bestParams))

[INFO] best score is 0.61 using {'learnRate': 0.0001, 'epochs': 30, 'dropout': 0.2, 'batch_size': 16}


In [74]:
searchResults.cv_results_["mean_test_score"]

array([0.42866905, 0.51683542, 0.43447156, 0.478973  , 0.49122492,
       0.44453645, 0.60982072, 0.52599544, 0.58212435, 0.42254148])

In [75]:
best_model=buildModel(learnRate=0.0001, dropout=0.2)

best_model.fit(x=X_train_permute.numpy(), y=Y_train.numpy(),
	validation_data=(X_test_permute.numpy(), Y_test.numpy()),
	batch_size=16,
	epochs=30)

Epoch 1/30
1441/1441 [==============================] - 15s 10ms/step - loss: 6.7587 - Accuracy: 0.5162 - recall_m: 1.0916 - precision_m: 0.5749 - f1_m: 0.7372 - val_loss: 6.4626 - val_Accuracy: 0.5189 - val_recall_m: 1.1620 - val_precision_m: 0.6087 - val_f1_m: 0.7831
Epoch 2/30
1441/1441 [==============================] - 14s 10ms/step - loss: 6.4524 - Accuracy: 0.5327 - recall_m: 1.2178 - precision_m: 0.6006 - f1_m: 0.7866 - val_loss: 7.1832 - val_Accuracy: 0.5363 - val_recall_m: 1.1718 - val_precision_m: 0.5583 - val_f1_m: 0.7421
Epoch 3/30
1441/1441 [==============================] - 14s 10ms/step - loss: 6.4574 - Accuracy: 0.5365 - recall_m: 1.2400 - precision_m: 0.5902 - f1_m: 0.7853 - val_loss: 7.0736 - val_Accuracy: 0.5633 - val_recall_m: 1.0054 - val_precision_m: 0.5814 - val_f1_m: 0.7172
Epoch 4/30
1441/1441 [==============================] - 14s 9ms/step - loss: 6.6885 - Accuracy: 0.5632 - recall_m: 1.1403 - precision_m: 0.5862 - f1_m: 0.7565 - val_loss: 6.1818 - val_Accura

In [76]:
loss, accuracy, recall, precision, f1score = best_model.evaluate(X_test_permute.numpy(), Y_test.numpy(), verbose=0)
print("Loss sur l'échantillon test: {}".format(round(loss,3)))
print("Accuracy sur l'échantillon test: {}".format(round(accuracy,3)))
print("Recall sur l'échantillon test: {}".format(round(recall,3)))
print("Precision sur l'échantillon test: {}".format(round(precision,3)))
print("F1-score sur l'échantillon test: {}".format(round(f1score,3)))

Loss sur l'échantillon test: 5.521
Accuracy sur l'échantillon test: 0.512
Recall sur l'échantillon test: 1.605
Precision sur l'échantillon test: 0.625
F1-score sur l'échantillon test: 0.895
